In [2]:
import os

def count_images_in_folder(folder_path):
    # Get all files in the folder
    files = os.listdir(folder_path)
    # Count image files (you can specify extensions like .jpg, .png, etc.)
    image_count = sum(1 for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')))
    return image_count

vehicle_dir = 'data4/non-vehicles'
non_vehicle_dir = 'data4/vehicles'
# Count images in each folder
vehicle_count = count_images_in_folder(vehicle_dir)
non_vehicle_count = count_images_in_folder(non_vehicle_dir)

# Print the results
print(f"Number of vehicle images: {vehicle_count}")
print(f"Number of non-vehicle images: {non_vehicle_count}")


Number of vehicle images: 8968
Number of non-vehicle images: 8792


In [4]:
import os
import numpy as np
from skimage.feature import hog
from skimage.io import imread
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from glob import glob
from random import sample
import matplotlib.pyplot as plt

# Step 1: Load and Limit Dataset
def load_limited_dataset(vehicle_dir, non_vehicle_dir, limit=75):
    # Load all vehicle and non-vehicle images
    vehicle_images = glob(vehicle_dir + '/*.png')
    non_vehicle_images = glob(non_vehicle_dir + '/*.png')
    
    # Randomly sample the specified limit from each category
    vehicle_images = sample(vehicle_images, min(limit, len(vehicle_images)))
    non_vehicle_images = sample(non_vehicle_images, min(limit, len(non_vehicle_images)))
    
    images = []
    labels = []
    
    # Load sampled vehicle images
    for file in vehicle_images:
        images.append(imread(file))
        labels.append(1)  # Label for vehicles
    
    # Load sampled non-vehicle images
    for file in non_vehicle_images:
        images.append(imread(file))
        labels.append(0)  # Label for non-vehicles
    
    return np.array(images), np.array(labels)

# Paths to your dataset folders
vehicle_dir = 'data4/non-vehicles'
non_vehicle_dir = 'data4/vehicles'

# Load limited dataset (1000 vehicles and 1000 non-vehicles)
images, labels = load_limited_dataset(vehicle_dir, non_vehicle_dir, limit=1000)

print(f"Loaded {len(images)} images: {sum(labels)} vehicles and {len(labels) - sum(labels)} non-vehicles")

# Step 2: Feature Extraction
def extract_hog_features(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    gray_image = rgb2gray(image)  # Convert to grayscale
    features = hog(
        gray_image,
        orientations=orientations,
        pixels_per_cell=pixels_per_cell,
        cells_per_block=cells_per_block,
        block_norm='L2-Hys',
        visualize=False
    )
    return features

def extract_features(images):
    feature_list = [extract_hog_features(image) for image in images]
    return np.array(feature_list)

# Extract features for all images
features = extract_features(images)

# Step 3: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Step 4: Train the SVM Classifier
svm = SVC(kernel='linear', C=1.0)
svm.fit(X_train, y_train)

# Step 5: Evaluate the Model
y_pred = svm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Loaded 2000 images: 1000 vehicles and 1000 non-vehicles
Accuracy: 0.96
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.94      0.96       201
           1       0.94      0.98      0.96       199

    accuracy                           0.96       400
   macro avg       0.96      0.96      0.96       400
weighted avg       0.96      0.96      0.96       400



# HOG AND SVM

In [39]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Function to load all images from a folder
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        if filepath.endswith(('.png')):
            img = cv2.imread(filepath)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (64, 64))  # Resize for uniformity
                images.append(img)
                labels.append(label)
    return images, labels

# Paths to your vehicle and non-vehicle folders
vehicle_folder = 'data4/vehicles'
no_vehicle_folder = "data4/non-vehicles"

# Load all images from both folders
vehicle_images, vehicle_labels = load_images_from_folder(vehicle_folder, 1)
no_vehicle_images, no_vehicle_labels = load_images_from_folder(no_vehicle_folder, 0)

# Combine the data
X = np.array(vehicle_images + no_vehicle_images)
y = np.array(vehicle_labels + no_vehicle_labels)

# Function to extract HOG features
def extract_hog_features(images):
    features = []
    for img in images:
        feature = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                      cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
        features.append(feature)
    return np.array(features)

# Extract HOG features for the dataset
X_hog = extract_hog_features(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.2, random_state=42)

# Train an SVM classifier
svm = LinearSVC(max_iter=10000)
svm.fit(X_train, y_train)

# Evaluate the model
y_pred = svm.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(classification_report(y_test, y_pred, target_names=["No Vehicle", "Vehicle"]))

# Save the trained model
joblib.dump(svm, "vehicle_detection_svm.pkl")

# Function to predict whether an image contains a vehicle


Accuracy: 95.75%
              precision    recall  f1-score   support

  No Vehicle       0.95      0.96      0.96      1801
     Vehicle       0.96      0.95      0.96      1751

    accuracy                           0.96      3552
   macro avg       0.96      0.96      0.96      3552
weighted avg       0.96      0.96      0.96      3552



['vehicle_detection_svm.pkl']

In [67]:
def predict_vehicle(image_path, svm):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (64, 64))
    hog_features = hog(resized, orientations=9, pixels_per_cell=(8, 8), 
                       cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    prediction = svm.predict([hog_features])
    return "Vehicle" if prediction[0] == 1 else "No Vehicle"

# Example prediction with a new image
test_image_path = "data4/full.png"
print(f"Prediction for {test_image_path}: {predict_vehicle(test_image_path, svm)}")


Prediction for data4/full.png: No Vehicle


# Decision Tree

In [16]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Function to load all images from a folder
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        if filepath.endswith(('.png')):
            img = cv2.imread(filepath)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (64, 64))  # Resize to uniform size
                images.append(img)
                labels.append(label)
    return images, labels

# Paths to your vehicle and non-vehicle folders
vehicle_folder = 'data4/vehicles'
no_vehicle_folder = "data4/non-vehicles"

# Load all images from both folders
vehicle_images, vehicle_labels = load_images_from_folder(vehicle_folder, 1)
no_vehicle_images, no_vehicle_labels = load_images_from_folder(no_vehicle_folder, 0)

# Combine the data
X = np.array(vehicle_images + no_vehicle_images)
y = np.array(vehicle_labels + no_vehicle_labels)

# Function to extract HOG features
def extract_hog_features(images):
    features = []
    for img in images:
        feature = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                      cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
        features.append(feature)
    return np.array(features)

# Extract HOG features for the dataset
X_hog = extract_hog_features(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.2, random_state=42)

In [17]:
# Train a Decision Tree classifier
tree_model = DecisionTreeClassifier(max_depth=10, random_state=42)
tree_model.fit(X_train, y_train)

# Evaluate the model
y_pred = tree_model.predict(X_test)
print(f"Decision Tree Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(classification_report(y_test, y_pred, target_names=["No Vehicle", "Vehicle"]))

# Save the trained model
joblib.dump(tree_model, "vehicle_detection_tree.pkl")

Decision Tree Accuracy: 90.43%
              precision    recall  f1-score   support

  No Vehicle       0.90      0.91      0.91      1801
     Vehicle       0.91      0.90      0.90      1751

    accuracy                           0.90      3552
   macro avg       0.90      0.90      0.90      3552
weighted avg       0.90      0.90      0.90      3552



['vehicle_detection_tree.pkl']

In [68]:
# Function to predict whether an image contains a vehicle
def predict_vehicle(image_path, tree_model):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (64, 64))
    hog_features = hog(resized, orientations=9, pixels_per_cell=(8, 8), 
                       cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    prediction = tree_model.predict([hog_features])
    return "Vehicle" if prediction[0] == 1 else "No Vehicle"

# Example prediction with a new image
test_image_path = "data4/full.png"
print(f"Prediction for {test_image_path}: {predict_vehicle(test_image_path, tree_model)}")

Prediction for data4/full.png: No Vehicle


In [29]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
data4/non-vehicles/image995.png

In [23]:
def load_images_from_folder(folder, label, img_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        if filepath.endswith(('.jpg', '.png', '.jpeg')):
            img = cv2.imread(filepath)
            if img is not None:
                img = cv2.resize(img, img_size)  # Resize for uniformity
                images.append(img)
                labels.append(label)
    return images, labels

# Paths to vehicle and non-vehicle folders
vehicle_folder = 'data4/vehicles'
no_vehicle_folder = "data4/non-vehicles"

# Load vehicle and non-vehicle data
vehicle_images, vehicle_labels = load_images_from_folder(vehicle_folder, 1)
no_vehicle_images, no_vehicle_labels = load_images_from_folder(no_vehicle_folder, 0)

# Combine and preprocess the dataset
X = np.array(vehicle_images + no_vehicle_images) / 255.0  # Normalize pixel values to [0, 1]
y = np.array(vehicle_labels + no_vehicle_labels)

# Reshape data for CNN (ensure images are in the format (height, width, channels))
X = X.reshape(-1, 64, 64, 3)

# Convert labels to categorical for binary classification
y = to_categorical(y, num_classes=2)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
# Build the CNN model
model = Sequential([
    # Convolutional layer 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolutional layer 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flatten the output of convolutional layers
    Flatten(),
    
    # Fully connected layers
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout for regularization
    Dense(2, activation='softmax')  # Output layer with 2 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping]
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the trained model
model.save("cnn_vehicle_detection.h5")


/Users/yashdode/Library/Python/3.12/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.8799 - loss: 0.2724 - val_accuracy: 0.9792 - val_loss: 0.0656
Epoch 2/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9807 - loss: 0.0608 - val_accuracy: 0.9783 - val_loss: 0.0563
Epoch 3/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9862 - loss: 0.0385 - val_accuracy: 0.9904 - val_loss: 0.0237
Epoch 4/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9911 - loss: 0.0277 - val_accuracy: 0.9887 - val_loss: 0.0313
Epoch 5/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.9933 - loss: 0.0212 - val_accuracy: 0.9947 - val_loss: 0.0160
Epoch 6/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9953 - loss: 0.0135 - val_accuracy: 0.9851 - val_loss: 0.0368
Epoch 7/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.9901 - loss: 0.0257 - val_accuracy: 0.9921 - val_loss: 0.0194
Epoch 8/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9942 - loss: 0.0175 - val_a

Test Accuracy: 99.47%


In [66]:
from tensorflow.keras.models import load_model

# Load the trained CNN model
model = load_model("cnn_vehicle_detection.h5")

def predict_vehicle(image_path, model):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (64, 64)) / 255.0  # Resize and normalize
    img = img.reshape(1, 64, 64, 3)  # Add batch dimension
    prediction = model.predict(img)
    return "Vehicle" if np.argmax(prediction) == 1 else "No Vehicle"

# Example prediction
test_image_path = "data4/full.png"
print(f"Prediction for {test_image_path}: {predict_vehicle(test_image_path, model)}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction for data4/full.png: No Vehicle


# CNN

In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Function to load images from a folder
def load_images_from_folder(folder, label, img_size=(128, 128)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        if filepath.endswith(('.jpg', '.png', '.jpeg')):
            img = cv2.imread(filepath)
            if img is not None:
                img = cv2.resize(img, img_size)  # Resize for uniformity
                images.append(img)
                labels.append(label)
    return images, labels

# Paths to vehicle and non-vehicle folders
vehicle_folder = 'data4/vehicles'
no_vehicle_folder = "data4/non-vehicles"


# Load vehicle and non-vehicle data
vehicle_images, vehicle_labels = load_images_from_folder(vehicle_folder, 1)
no_vehicle_images, no_vehicle_labels = load_images_from_folder(no_vehicle_folder, 0)

# Combine and preprocess the dataset
X = np.array(vehicle_images + no_vehicle_images) / 255.0  # Normalize pixel values to [0, 1]
y = np.array(vehicle_labels + no_vehicle_labels)

# Reshape data for CNN
X = X.reshape(-1, 128, 128, 3)  # Adjust input dimensions
y = to_categorical(y, num_classes=2)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)
datagen.fit(X_train)

# Option 1: CNN from Scratch
def cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu', kernel_regularizer='l2'),  # L2 regularization
        Dropout(0.5),  # Dropout for regularization
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Option 2: Transfer Learning with MobileNetV2
def mobilenet_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    for layer in base_model.layers:
        layer.trainable = False  # Freeze pre-trained layers

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Choose your model
use_transfer_learning = True  # Set to False for CNN from scratch
if use_transfer_learning:
    model = mobilenet_model()
    print("Using MobileNetV2 for Transfer Learning...")
else:
    model = cnn_model()
    print("Using Custom CNN from Scratch...")

# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=20,
    callbacks=[early_stopping]
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the trained model
model.save("vehicle_detection_model.keras")

Using MobileNetV2 for Transfer Learning...
Epoch 1/20


/Users/yashdode/Library/Python/3.12/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


444/444 ━━━━━━━━━━━━━━━━━━━━ 28s 60ms/step - accuracy: 0.9258 - loss: 0.1856 - val_accuracy: 0.9868 - val_loss: 0.0364
Epoch 2/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 28s 62ms/step - accuracy: 0.9739 - loss: 0.0718 - val_accuracy: 0.9885 - val_loss: 0.0335
Epoch 3/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 30s 67ms/step - accuracy: 0.9732 - loss: 0.0675 - val_accuracy: 0.9876 - val_loss: 0.0299
Epoch 4/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 31s 71ms/step - accuracy: 0.9777 - loss: 0.0588 - val_accuracy: 0.9921 - val_loss: 0.0246
Epoch 5/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 34s 76ms/step - accuracy: 0.9789 - loss: 0.0603 - val_accuracy: 0.9966 - val_loss: 0.0132
Epoch 6/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 34s 77ms/step - accuracy: 0.9799 - loss: 0.0520 - val_accuracy: 0.9947 - val_loss: 0.0178
Epoch 7/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 35s 80ms/step - accuracy: 0.9853 - loss: 0.0442 - val_accuracy: 0.9913 - val_loss: 0.0287
Epoch 8/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 35s 80ms/step - accuracy: 0.9831 - loss: 0.0455 - val_accurac

In [3]:
# Function to predict a new image
def predict_vehicle(image_path, model):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128)) / 255.0  # Resize and normalize
    img = img.reshape(1, 128, 128, 3)  # Add batch dimension
    prediction = model.predict(img)
    return "Vehicle" if np.argmax(prediction) == 1 else "No Vehicle"

# Example prediction with a new image
test_image_path = "data4/full.png"
print(f"Prediction for {test_image_path}: {predict_vehicle(test_image_path, model)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
Prediction for data4/full.png: Vehicle


In [54]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Function to preprocess an image
def preprocess_image(filepath, img_size=(64, 64)):
    """
    Preprocesses an image:
    1. Reads and converts to grayscale.
    2. Resizes to img_size.
    3. Applies histogram equalization.
    4. Applies Gaussian blur.
    5. Normalizes pixel values to [0, 1].
    """
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    img = cv2.resize(img, img_size)  # Resize
    img = cv2.equalizeHist(img)  # Histogram equalization
    img = cv2.GaussianBlur(img, (3, 3), 0)  # Gaussian Blur
    img = img / 255.0  # Normalize pixel values
    return img

# Function to load and preprocess images from a folder
def load_images_from_folder(folder, label, img_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        if filepath.endswith(('.png')):
            img = preprocess_image(filepath, img_size)
            images.append(img)
            labels.append(label)
    return images, labels

# Paths to vehicle and non-vehicle folders
vehicle_folder = 'data4/vehicles'
no_vehicle_folder = "data4/non-vehicles"

# Load vehicle and non-vehicle data
vehicle_images, vehicle_labels = load_images_from_folder(vehicle_folder, 1)
no_vehicle_images, no_vehicle_labels = load_images_from_folder(no_vehicle_folder, 0)

# Combine the data
X = np.array(vehicle_images + no_vehicle_images)
y = np.array(vehicle_labels + no_vehicle_labels)

# Extract HOG features
def extract_hog_features(images):
    features = []
    for img in images:
        feature = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                      cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
        features.append(feature)
    return np.array(features)

# Extract HOG features for the entire dataset
X_hog = extract_hog_features(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.2, random_state=42)

In [ ]:
# Train the SVM model
svm = LinearSVC(max_iter=10000)
svm.fit(X_train, y_train)

# Evaluate the model
y_pred = svm.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(classification_report(y_test, y_pred, target_names=["No Vehicle", "Vehicle"]))

# Save the trained model
joblib.dump(svm, "svm_vehicle_detection.pkl")

# Function to predict a single image
def predict_vehicle(image_path, svm, img_size=(64, 64)):
    img = preprocess_image(image_path, img_size)
    hog_features = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                       cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    prediction = svm.predict([hog_features])
    return "Vehicle" if prediction[0] == 1 else "No Vehicle"

In [64]:
# Example prediction
test_image_path = "data4/full.png"
print(f"Prediction for {test_image_path}: {predict_vehicle(test_image_path, svm)}")

Prediction for data4/full.png: No Vehicle


In [ ]:

'''
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained deep learning model
model = load_model("vehicle_detection_model.keras")  # Replace with your model file

def sliding_window(image, step_size, window_size):
    """
    Slide a window across the image.
    :param image: Input image.
    :param step_size: Number of pixels to move the window.
    :param window_size: Tuple (width, height) for the window size.
    :return: Generator yielding (x, y, window).
    """
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

def predict_and_draw_boxes(image_path, model, window_size=(64, 64), step_size=32, threshold=0.5):
    """
    Detect vehicles in an image and draw bounding boxes.
    :param image_path: Path to input image.
    :param model: Trained deep learning model.
    :param window_size: Size of sliding window.
    :param step_size: Step size for sliding window.
    :param threshold: Prediction confidence threshold.
    """
    # Load and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (512, 512))  # Resize for faster processing
    orig_image = image.copy()
    image = image / 255.0  # Normalize pixel values
    
    # List to store bounding boxes
    boxes = []
    
    # Sliding window detection
    for (x, y, window) in sliding_window(image, step_size, window_size):
        if window.shape[0] != window_size[1] or window.shape[1] != window_size[0]:
            continue  # Ignore windows that are not the correct size
        
        # Preprocess window
        window = window.reshape(1, window_size[1], window_size[0], 3)  # Reshape for the model
        
        # Predict using the model
        prediction = model.predict(window, verbose=0)
        confidence = prediction[0][1]  # Confidence for 'Vehicle' class
        
        # If confidence exceeds threshold, save the bounding box
        if confidence > threshold:
            boxes.append((x, y, x + window_size[0], y + window_size[1]))
    
    # Draw bounding boxes on the original image
    for (x1, y1, x2, y2) in boxes:
        cv2.rectangle(orig_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green rectangle
    
    # Display the image with bounding boxes
    cv2.imshow("Vehicle Detection", orig_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # Optionally save the image with bounding boxes
    cv2.imwrite("vehicle_detection_output.jpg", orig_image)
    print("Output image saved as 'data4/output.png'.")

# Path to the test image
test_image_path = "data4/carfull.png"  # Replace with your test image path

# Run vehicle detection
# Run vehicle detection with correct window size
predict_and_draw_boxes(test_image_path, model, window_size=(128, 128), step_size=32, threshold=0.7)


2024-12-16 00:09:02.211 Python[80512:4979407] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [ ]:
'''

import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained deep learning model
model = load_model("vehicle_detection_model.keras")  # Replace with your model file path

def predict_whole_image(image_path, model, threshold=0.5):
    """
    Detect vehicles using a single bounding box assumption.
    :param image_path: Path to the input image.
    :param model: Trained deep learning model.
    :param threshold: Confidence threshold for vehicle detection.
    """
    # Load and preprocess the image
    image = cv2.imread(image_path)
    orig_image = image.copy()
    
    # Resize to match model input size
    input_size = (128, 128)
    resized_image = cv2.resize(image, input_size)
    resized_image = resized_image / 255.0  # Normalize pixel values
    resized_image = resized_image.reshape(1, input_size[0], input_size[1], 3)  # Add batch dimension
    
    # Predict using the model
    prediction = model.predict(resized_image, verbose=0)
    confidence = prediction[0][1]  # Confidence score for 'Vehicle' class
    
    # Draw a single bounding box if confidence exceeds threshold
    if confidence > threshold:
        height, width = orig_image.shape[:2]
        x1, y1 = int(width * 0.2), int(height * 0.2)  # Adjust the box size as needed
        x2, y2 = int(width * 0.8), int(height * 0.8)
        
        # Draw a green rectangle around the car
        cv2.rectangle(orig_image, (x1, y1), (x2, y2), (0, 255, 0), 3)
        label = f"Vehicle ({confidence*100:.2f}%)"
        cv2.putText(orig_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Display the output image
    cv2.imshow("Detected Vehicle", orig_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the output image
    cv2.imwrite("vehicle_detection_single_box.jpg", orig_image)
    print("Output image saved as 'vehicle_detection_single_box.jpg'.")

# Path to the test image
test_image_path = "data4/carfull2.png"  # Replace with your test image path


# Run detection
predict_whole_image(test_image_path, model, threshold=0.7)


2024-12-16 00:09:28.875 Python[80518:4984002] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


# Using CNN boxes

In [2]:
pip install imutils


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25837 sha256=512ca747bf094d3a865455306495caf4a97e1644b064ac2eb20c4ebc33b3eba7
  Stored in directory: /Users/yashdode/Library/Caches/pip/wheels/5b/76/96/ad0c321506837bef578cf3008df3916c23018435a355d9f6b1
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from imutils.object_detection import non_max_suppression

# Load the trained deep learning model
model = load_model("vehicle_detection_model.keras")  # Replace with your model file path

def sliding_window(image, step_size, window_size):
    """
    Slide a window across the image.
    :param image: Input image.
    :param step_size: Number of pixels to move the window.
    :param window_size: Tuple (width, height) for the window size.
    :return: Generator yielding (x, y, window).
    """
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

def multi_scale_detection(image_path, model, window_size=(128, 128), step_size=32, scales=[1.0, 1.5, 2.0], threshold=0.7):
    """
    Detect vehicles in an image using sliding windows at multiple scales and apply NMS.
    Ensures only the single most confident bounding box is drawn.
    """
    # Load the image
    image = cv2.imread(image_path)
    orig_image = image.copy()
    boxes = []  # List to store bounding boxes and confidence scores

    # Multi-scale sliding window
    for scale in scales:
        resized_image = cv2.resize(image, None, fx=1/scale, fy=1/scale)
        for (x, y, window) in sliding_window(resized_image, step_size, window_size):
            if window.shape[0] != window_size[1] or window.shape[1] != window_size[0]:
                continue  # Skip if the window is not the correct size
            
            # Preprocess the window for the model
            window_resized = window / 255.0
            window_resized = window_resized.reshape(1, window_size[1], window_size[0], 3)
            
            # Predict using the model
            prediction = model.predict(window_resized, verbose=0)
            confidence = prediction[0][1]  # Confidence for 'Vehicle' class
            
            # Save the bounding box if confidence exceeds threshold
            if confidence > threshold:
                x1 = int(x * scale)
                y1 = int(y * scale)
                x2 = int((x + window_size[0]) * scale)
                y2 = int((y + window_size[1]) * scale)
                boxes.append((x1, y1, x2, y2, confidence))

    # Apply Non-Maximum Suppression (NMS)
    rects = np.array([[x1, y1, x2, y2] for (x1, y1, x2, y2, _) in boxes])
    scores = np.array([conf for (_, _, _, _, conf) in boxes])
    selected_boxes = non_max_suppression(rects, probs=scores, overlapThresh=0.3)

    # Select the single most confident bounding box
    if len(selected_boxes) > 0:
        best_box = selected_boxes[0]  # First box after NMS (highest confidence)
        x1, y1, x2, y2 = best_box
        cv2.rectangle(orig_image, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(orig_image, "Vehicle", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the output image
    cv2.imshow("Vehicle Detection", orig_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the output image
    cv2.imwrite("vehicle_detection_single_box.jpg", orig_image)
    print("Output image saved as 'vehicle_detection_single_box.jpg'.")

# Path to the test image
test_image_path = "data4/carfull3.png"  # Replace with your test image path

# Run vehicle detection
multi_scale_detection(test_image_path, model, window_size=(128, 128), step_size=64, scales=[1.0, 1.5, 2.0], threshold=0.7)


2024-12-16 00:17:39.947 Python[80599:5010670] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from imutils.object_detection import non_max_suppression

# Load the trained deep learning model
model = load_model("vehicle_detection_model.keras")  # Replace with your model file path

def sliding_window(image, step_size, window_size):
    """
    Slide a rectangular window across the image.
    :param image: Input image.
    :param step_size: Number of pixels to move the window.
    :param window_size: Tuple (width, height) for the window size.
    :return: Generator yielding (x, y, window).
    """
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            yield (x, y, window_size[0], window_size[1], image[y:y + window_size[1], x:x + window_size[0]])

def multi_scale_detection(image_path, model, window_sizes=[(192, 128), (256, 128)], step_size=64, threshold=0.8):
    """
    Detect vehicles in an image using sliding rectangular windows and apply NMS.
    Ensures only the single most confident bounding box is drawn.
    """
    # Load the image
    image = cv2.imread(image_path)
    orig_image = image.copy()
    boxes = []  # List to store bounding boxes and confidence scores

    # Sliding windows at multiple sizes
    for window_size in window_sizes:
        for (x, y, w, h, window) in sliding_window(image, step_size, window_size):
            if window.shape[0] != h or window.shape[1] != w:
                continue  # Skip invalid windows
            
            # Preprocess the window for the model
            window_resized = cv2.resize(window, (128, 128)) / 255.0
            window_resized = window_resized.reshape(1, 128, 128, 3)
            
            # Predict using the model
            prediction = model.predict(window_resized, verbose=0)
            confidence = prediction[0][1]  # Confidence for 'Vehicle' class
            
            # Save the bounding box if confidence exceeds threshold
            if confidence > threshold:
                x1, y1, x2, y2 = x, y, x + w, y + h
                boxes.append((x1, y1, x2, y2, confidence))

    # Apply Non-Maximum Suppression (NMS)
    rects = np.array([[x1, y1, x2, y2] for (x1, y1, x2, y2, _) in boxes])
    scores = np.array([conf for (_, _, _, _, conf) in boxes])
    selected_boxes = non_max_suppression(rects, probs=scores, overlapThresh=0.3)

    # Select the single most confident bounding box
    if len(selected_boxes) > 0:
        # Draw only the box with the highest confidence
        best_box_index = np.argmax(scores)
        x1, y1, x2, y2 = rects[best_box_index]
        cv2.rectangle(orig_image, (x1, y1), (x2, y2), (0, 255, 0), 3)
        label = "Vehicle"
        cv2.putText(orig_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the output image
    cv2.imshow("Refined Vehicle Detection", orig_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the output image
    cv2.imwrite("refined_vehicle_single_box.jpg", orig_image)
    print("Output image saved as 'refined_vehicle_single_box.jpg'.")

# Path to the test image
test_image_path = "data4/carfull3.png"  # Replace with your test image path

# Run refined vehicle detection
multi_scale_detection(test_image_path, model, window_sizes=[(192, 128), (256, 128)], step_size=64, threshold=0.8)


2024-12-16 00:28:25.467 Python[80709:5065311] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [1]:
pip install torch torchvision matplotlib opencv-python


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install yolov5


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# YOLO 

In [ ]:
import cv2
import torch
import numpy as np

# Load YOLOv5 Model (pre-trained on COCO dataset)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # 'yolov5s' is the small model; you can use 'yolov5m' or 'yolov5l' for better accuracy

def detect_vehicles_yolo(image_path, save_output=True):
    """
    Detect vehicles in an image using YOLOv5.
    :param image_path: Path to the input image.
    :param save_output: Whether to save the output image.
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found.")
        return
    height, width, _ = image.shape

    # Perform inference using YOLO
    results = model(image)

    # Filter results to include only 'car', 'truck', or 'bus' classes
    filtered_results = results.pandas().xyxy[0]  # Bounding boxes in pandas DataFrame format
    vehicle_classes = ['car', 'truck', 'bus', 'motorbike']
    filtered_results = filtered_results[filtered_results['name'].isin(vehicle_classes)]

    # Draw bounding boxes on the image
    for _, row in filtered_results.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        label = f"{row['name']} {row['confidence']:.2f}"
        color = (0, 255, 0)  # Green bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Display the output image
    cv2.imshow("YOLO Vehicle Detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the output image
    if save_output:
        output_path = "yolo_vehicle_detection.jpg"
        cv2.imwrite(output_path, image)
        print(f"Output image saved as {output_path}.")

# Path to your test image
image_path = "data4/carfull3.png"  # Replace with your image path

# Run vehicle detection
detect_vehicles_yolo(image_path)


Using cache found in /Users/yashdode/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-1 Python-3.12.0 torch-2.2.2 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
2024-12-16 00:29:06.796 Python[80723:5071172] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
